### 3 - Etapa de processamento dos dados

Nesta etapa faremos a **seleção de variáveis**, **seleção de algoritmos**, **seleção de modelos de aprendizagem**, **seleção de métricas de avaliação** e aplicaremos técnicas de amostragem para estabelecer a divisão dos **dados de treino** e **dados de teste** dos algoritmos.

In [1]:
# Pacotes utilizados 
import pandas as pd 
import numpy as np 
import seaborn as sea
import matplotlib.pyplot as plt
%matplotlib inline

## Normalização 
from sklearn.preprocessing import MinMaxScaler
## Padronização 
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

# Métricas 

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

# Algoritmos de aprendizagem 

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

import warnings
warnings.filterwarnings("ignore")

In [2]:
#### Dataframe 

df = pd.read_csv('dados/df_bruto.csv')
df_transf = pd.read_csv('dados/df_transf.csv')

In [3]:
df_transf.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0


#### 3.1) Tratamento dos dados 

#### 3.2) Seleção de variáveis

#### 3.2) Seleção de algoritmos e métricas

#### Métricas utilizadas

Para avaliação de todos os modelos utilizaremos as três métricas descritas abaixo, um modelo que tiver avaliações melhores em **duas ou mais** métricas será o modelo escolhido, para todas as comparações. 

**R²**: calcula qual a porcentagem da variância que pode ser prevista pelo modelo, por exemplo, R² = XX significa que XX% da variância dos dados pode ser explicada pelo modelo. Essa métrica varia de 0 a 1 e quanto mais próximo de 1, melhor é o modelo. 

**MAPE**: exprime uma porcentagem, significando que, em média, o modelo faz previsões que erram por XX% do valor real, ou seja, quanto menor o MAPE, melhor é o modelo. 

**RMSE**: entra como uma forma de melhorar a interpretabilidade da métrica, pois o erro médio possui a mesma unidade da variável alvo, assim quanto menor o RMSE, melhor é o modelo.  

In [4]:
# Função para aplicar as métricas 

def metricas(y_teste, y_pred):
    rmse = (mean_squared_error(y_teste, y_pred))**(1/2)
    print("R² = %.1f %%" % (r2_score(y_teste, y_pred)*100))
    print("MAPE = %.1f %% " % (mean_absolute_percentage_error(y_teste, y_pred)*100))
    print("RMSE = %.1f USD" % (rmse)) 
    
def valores_metricas(y_teste, y_pred):
    r2 = (r2_score(y_teste, y_pred)*100)
    mape = (mean_absolute_percentage_error(y_teste, y_pred)*100)
    rmse = (mean_squared_error(y_teste, y_pred))**(1/2)
    return [r2, mape, rmse]

#### Modelo base 

O **modelo base** utilizará o algoritmo de **regressão linear multivariada** e o conjunto de dados sem nenhum tipo de tratamento de forma, outliers ou seleção de variáveis, ou seja, será treinado com o **dataframe pré-processado** (sem valores missing e com variáveis transformadas), utilizando **amostragem estática (train_test_split)**. 

In [5]:
# Divide os dados em treino e teste

variaveis = list(df_transf.columns)
variaveis.remove('SalePrice')

X = df_transf[variaveis].values
Y = df_transf['SalePrice'].values


x_treino, x_teste, y_treino, y_teste = train_test_split(X, Y, test_size = 0.30, random_state = 5)

# Criando e treinando o modelo

modelo = LinearRegression(normalize = False).fit(x_treino, y_treino)
y_pred = modelo.predict(x_teste)

metricas(y_teste, y_pred)

R² = 68.5 %
MAPE = 11.5 % 
RMSE = 42971.9 USD


In [26]:
# Preparando a lista de modelos

def selec_model(X, Y):

    x_treino, x_teste, y_treino, y_teste = train_test_split(X, Y, test_size = 0.30, random_state = 5)

    modelos = []
    #modelos.append(('LR', LinearRegression()))
    modelos.append(('LASSO', Lasso()))
    modelos.append(('RIDGE', Ridge()))
    modelos.append(('ELASTIC', ElasticNet()))
    modelos.append(('KNN_R', KNeighborsRegressor()))
    modelos.append(('CART', DecisionTreeRegressor()))
    modelos.append(('SVR', SVR()))

    resultados = {}
    for nome, modelo in modelos:
        model = modelo.fit(x_treino, y_treino)
        y_pred = model.predict(x_teste)
        resultados[nome] = valores_metricas(y_teste, y_pred)

    df_result = pd.DataFrame(resultados, index = ['R²','MAPE','RMSE'])
    return df_result.round(1)

selec_model(X, Y)

,LASSO,RIDGE,ELASTIC,KNN_R,CART,SVR
R²,70.2,70.8,64.6,61.1,72.5,-6.1
MAPE,10.9,10.7,11.8,16.8,14.3,27.9
RMSE,41774.5,41336.3,45550.7,47726.0,40098.8,78813.6


#### Normalizando e/ou Padronizando.

Normalizando apenas.

In [27]:
variaveis = list(df_transf.columns)
variaveis.remove('SalePrice')

X = df_transf[variaveis].values
Y = df_transf['SalePrice'].values

Xnorm = MinMaxScaler().fit_transform(X)

selec_model(X, Y)

,LASSO,RIDGE,ELASTIC,KNN_R,CART,SVR
R²,70.2,70.8,64.6,61.1,74.2,-6.1
MAPE,10.9,10.7,11.8,16.8,13.9,27.9
RMSE,41774.5,41336.3,45550.7,47726.0,38879.4,78813.6


Padronizando apenas.

In [28]:
variaveis = list(df_transf.columns)
variaveis.remove('SalePrice')

X = df_transf[variaveis].values
Y = df_transf['SalePrice'].values

Xpad = StandardScaler().fit_transform(X)

selec_model(Xpad, Y)

,LASSO,RIDGE,ELASTIC,KNN_R,CART,SVR
R²,69.4,70.3,76.4,63.9,67.1,-6.1
MAPE,11.4,11.1,9.4,14.2,14.7,27.9
RMSE,42329.8,41683.6,37181.1,45943.3,43909.0,78799.7


Normalizando e Padronizando.

In [29]:
variaveis = list(df_transf.columns)
variaveis.remove('SalePrice')

X = df_transf[variaveis].values
Y = df_transf['SalePrice'].values

Xnorm_pad = StandardScaler().fit_transform(Xnorm)

selec_model(Xnorm_pad, Y)

,LASSO,RIDGE,ELASTIC,KNN_R,CART,SVR
R²,69.4,70.3,76.4,63.9,74.8,-6.1
MAPE,11.4,11.1,9.4,14.2,14.3,27.9
RMSE,42329.8,41683.6,37181.1,45943.3,38420.2,78799.7


Padronizando e Normalizando.

In [30]:
variaveis = list(df_transf.columns)
variaveis.remove('SalePrice')

X = df_transf[variaveis].values
Y = df_transf['SalePrice'].values

Xpad_norm = MinMaxScaler().fit_transform(Xpad)

selec_model(Xpad_norm, Y)

,LASSO,RIDGE,ELASTIC,KNN_R,CART,SVR
R²,70.3,74.9,62.8,64.1,60.0,-6.0
MAPE,10.9,10.4,15.0,14.5,15.0,27.9
RMSE,41723.1,38319.0,46647.0,45870.9,48364.8,78786.3


Com base nos resultados concluímos que os melhores algoritmos em ordem, para cada caso foram:

Normalizados: **CART**, **RIDGE**

Padronizados: **ELASTIC**, **RIDGE** 

Para dados normalizados e padronizados: **ELASTIC**, **RIDGE** 

Para dados padronizados e normalizados : **RIDGE**, **LASSO**

#### 3.3) Seleção de modelos 